In [1]:
import pandas as pd
import pickle
from IPython.display import display, HTML
import numpy as np
import ast
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from collections import Counter
from numpy.linalg import norm
import plotly.graph_objects as go

In [2]:
def obtener_clusters_ordenados(etiquetas):
    # Contar el número de documentos por clúster
    conteo_clusters = Counter(etiquetas)

    # Eliminar el clúster residual (-1)
    #del conteo_clusters[-1]

    # Ordenar por número de documentos (de mayor a menor)
    clusters_ordenados = sorted(conteo_clusters.items(), key=lambda x: x[1], reverse=True)
    return clusters_ordenados

def filter_documents_for_dictionary(tokenized_documents, filtered_words):
    filtered_documents = []
    for doc in tokenized_documents:
        filtered_doc = [word for word in doc if word in filtered_words]
        filtered_documents.append(filtered_doc)
    return filtered_documents

In [3]:
with open('etiquetas_meta_HDBSCAN.pkl', 'rb') as archivo:
    etiquetas_meta_documentos = pickle.load(archivo)
df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=";")
dictionary = Dictionary.load('dictionary_filtrado_spacy.gensim')
filtered_words = set(dictionary.token2id.keys())
preprocesado_documents=df_preprocesado['texto_preprocesado'].apply(ast.literal_eval)
# Generar documentos en igual de condiciones
documents = filter_documents_for_dictionary(preprocesado_documents,filtered_words)
modelo_w2v_cargado = Word2Vec.load("w2v_250_epochs.model")
documentos_originales=df_preprocesado['texto']
clusters_ordenados = obtener_clusters_ordenados(etiquetas_meta_documentos)
cluster_sizes_dict = dict(clusters_ordenados)

In [4]:
def obtener_palabras_representativas_w2v2_con_pesos_normalizados(model_w2v, documents, labels, n=10):
    cluster_palabras_representativas = {}

    for cluster_label in np.unique(labels):
        cluster_indices = np.where(labels == cluster_label)[0]
        cluster_documentos = [documents[i] for i in cluster_indices]

        # Obtener embeddings promedio de palabras por documento
        cluster_vectores = []
        for doc in cluster_documentos:
            palabra_vectors = [model_w2v.wv[palabra] for palabra in doc if palabra in model_w2v.wv]
            if palabra_vectors:  # Evitar documentos vacíos
                cluster_vectores.append(np.mean(palabra_vectors, axis=0))

        if not cluster_vectores:
            cluster_palabras_representativas[cluster_label] = []
            continue

        # Calcular el promedio del clúster
        cluster_centroide = np.mean(cluster_vectores, axis=0)

        # Encontrar palabras más similares al promedio
        try:
            palabras_cercanas = model_w2v.wv.similar_by_vector(cluster_centroide, topn=n)
            palabras_con_pesos = []

            # Calcular los pesos proporcionalmente a la proximidad al centroide
            for palabra, score in palabras_cercanas:
                peso = score  # El peso puede ser directamente la similitud
                palabras_con_pesos.append((palabra, peso))

            # Normalizar los pesos para que su suma sea igual al tamaño del clúster
            total_peso = sum(peso for _, peso in palabras_con_pesos)
            total_documentos = len(cluster_documentos)  # Tamaño del clúster
            palabras_con_pesos_dobles = [(palabra, peso, (peso / total_peso) * total_documentos) for palabra, peso in palabras_cercanas]
            cluster_palabras_representativas[cluster_label] = palabras_con_pesos_dobles

        except Exception as e:
            cluster_palabras_representativas[cluster_label] = []

    return cluster_palabras_representativas


In [5]:
def construir_documentos_representativos_para_treemap_ponderado(
    cluster_keywords_weights,
    etiquetas_predichas,
    documentos_preprocesados,
    documentos_originales,
    modelo_w2v,
    num_documentos=5
):
    documentos_representativos = {}

    for cluster_id, lista_palabras in cluster_keywords_weights.items():
        documentos_representativos[cluster_id] = {}

        for palabra, peso_real, peso_palabra in lista_palabras:
            if palabra not in modelo_w2v.wv:
                continue

            vector_objetivo = modelo_w2v.wv[palabra]
            docs_similares = []

            for i, (etiqueta, doc) in enumerate(zip(etiquetas_predichas, documentos_preprocesados)):
                if etiqueta != cluster_id:
                    continue

                vectores_doc = [modelo_w2v.wv[p] for p in doc if p in modelo_w2v.wv]
                if not vectores_doc:
                    continue

                vector_promedio = np.mean(vectores_doc, axis=0)
                similitud = np.dot(vector_promedio, vector_objetivo) / (
                    norm(vector_promedio) * norm(vector_objetivo)
                )

                doc_resumen = documentos_originales[i][:100] + "..."
                docs_similares.append((doc_resumen, similitud))

            # Ordenar y limitar a los documentos más similares
            docs_similares = sorted(docs_similares, key=lambda x: x[1], reverse=True)[:num_documentos]

            # Normalizar similitudes para que sumen 1 y repartir el peso_palabra
            suma_similitudes = sum(sim for _, sim in docs_similares)
            if suma_similitudes > 0:
                docs_ponderados = [
                    (doc, sim,(sim / suma_similitudes) * peso_palabra)
                    for doc, sim in docs_similares
                ]
                documentos_representativos[cluster_id][palabra] = docs_ponderados

    return documentos_representativos


In [6]:
def exportar_treemap_clusters_palabras_documentos_go_tooltip(
    cluster_sizes,
    cluster_keywords_weights,
    documentos_representativos,
    output_filename="treemap_clusters_palabras_documentos.html",
    nombres_clusters=None  # Nuevo parámetro opcional
):
    labels = []
    parents = []
    values = []
    hover_texts = []

    for cluster_id, size in cluster_sizes.items():
        # Usar nombre representativo si está disponible, si no, usar "Cluster {id}"
        cluster_name = nombres_clusters.get(cluster_id, f"Cluster {cluster_id}") if nombres_clusters else f"Cluster {cluster_id}"
        
        labels.append(cluster_name)
        parents.append("")
        values.append(size)
        hover_texts.append(f"{cluster_name}<br>Tamaño: {size}")

        # Palabras clave dentro del clúster
        for palabra, peso_real, peso_normalizado in cluster_keywords_weights[cluster_id]:
            palabra_id = f"{palabra} ({cluster_name})"
            labels.append(palabra_id)
            parents.append(cluster_name)
            values.append(peso_normalizado)
            hover_texts.append(f"Palabra: {palabra}<br>Relevancia: {peso_real:.2f}")

            # Documentos representativos de esa palabra
            if cluster_id in documentos_representativos and palabra in documentos_representativos[cluster_id]:
                for doc, peso_real_doc, peso_norm_doc in documentos_representativos[cluster_id][palabra]:
                    doc_id = f"{doc} ({palabra})"
                    labels.append(doc_id)
                    parents.append(palabra_id)
                    values.append(peso_norm_doc)
                    hover_texts.append(f"<b>Texto:</b><br>{doc}"
                                      f"<b>Relevancia:</b> {peso_real_doc:.3f}")

    fig = go.Figure(go.Treemap(
        labels=labels,
        parents=parents,
        values=values,
        branchvalues="remainder",
        hovertext=hover_texts,
        hoverinfo="text"
    ))

    fig.update_layout(title_text="Modelo de tópicos", title_x=0.5)
    fig.write_html(output_filename)
    print(f"Treemap exportado como {output_filename}")


In [7]:
def reasignar_clusters_por_tamaño(cluster_sizes_ordenado, cluster_keywords_weights):
    # Paso 1: Crear el nuevo mapeo de etiquetas antiguas a nuevas, en orden decreciente de tamaño
    claves_ordenadas = list(cluster_sizes_ordenado.keys())
    mapeo_clusters = {clave_antigua: nueva_id for nueva_id, clave_antigua in enumerate(claves_ordenadas)}
    
    # Paso 2: Aplicar el mapeo a cluster_keywords_weights
    nuevo_cluster_keywords_weights = {
        mapeo_clusters[cluster_id]: palabras
        for cluster_id, palabras in cluster_keywords_weights.items()
        if cluster_id in mapeo_clusters
    }
    
    return nuevo_cluster_keywords_weights, mapeo_clusters


In [8]:
def reasignar_documentos_representativos(documentos_representativos, mapeo_clusters):
    """
    Reasigna los IDs de cluster en el diccionario de documentos representativos
    usando el mapeo de IDs antiguos a nuevos.
    
    Args:
        documentos_representativos (dict): Diccionario con claves de cluster originales.
        mapeo_clusters (dict): Diccionario que mapea ID antiguo → ID nuevo.

    Returns:
        dict: Nuevo diccionario con las claves reasignadas.
    """
    nuevos_documentos = {
        mapeo_clusters[cluster_id]: contenido
        for cluster_id, contenido in documentos_representativos.items()
        if cluster_id in mapeo_clusters
    }
    return nuevos_documentos


In [9]:
cluster_keywords_weights = obtener_palabras_representativas_w2v2_con_pesos_normalizados(
    model_w2v=modelo_w2v_cargado,
    documents=documents,
    labels=np.array(etiquetas_meta_documentos),
    n=10
  
)


In [10]:
documentos_representativos = construir_documentos_representativos_para_treemap_ponderado(
    cluster_keywords_weights,
    etiquetas_meta_documentos,
    documents,
    documentos_originales,
    modelo_w2v=modelo_w2v_cargado
)


In [11]:
nuevo_cluster_keywords_weights, mapeo_clusters = reasignar_clusters_por_tamaño(
    cluster_sizes_dict, cluster_keywords_weights
)




In [12]:
nuevos_documentos_representativos = reasignar_documentos_representativos(
    documentos_representativos, mapeo_clusters
)


In [13]:
cluster_sizes_ordenado = {i: valor for i, (_, valor) in enumerate(list(cluster_sizes_dict.items())[:13])}
cluster_sizes = cluster_sizes_ordenado
cluster_keywords_weights=nuevo_cluster_keywords_weights
documentos_representativos=nuevos_documentos_representativos
nombres_clusters = {
    0: "Varios tipos Incidencias vía pública",
    1: "Estado de la vía pública ",
    2: "Limpieza",
    3: "Tráfico",
    4: "Trámites y servicios",
    5: "Juegos infantiles",
    6: "Transporte público",
    7: "Parques y jardines",
    8: "Instalaciones municipales",
    9: "Urbanismo"

    
}

exportar_treemap_clusters_palabras_documentos_go_tooltip(
    cluster_sizes,
    cluster_keywords_weights,
    documentos_representativos,
    output_filename="treemap_temasv3.html",
    nombres_clusters=nombres_clusters
)


Treemap exportado como treemap_temasv3.html
